# Clone repository from github, mount Google Drive for data, install libs

In [ ]:
!git clone https://github.com/Intelligent-Systems-Phystech/2022-Project-94

Cloning into '2022-Project-94'...
remote: Enumerating objects: 697, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 697 (delta 62), reused 52 (delta 26), pack-reused 604
Receiving objects: 100% (697/697), 1.94 GiB | 13.77 MiB/s, done.
Resolving deltas: 100% (258/258), done.


In [ ]:
import os
os.rename("2022-Project-94", "HailProject")

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pip install cfgrib
!pip install eccodes 
!pip install ecmwflibs
!pip install xarray
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 2.0 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
  Created wheel for eccodes: filename=eccodes-1.4.2-py3-none-any.whl size=39817 sha256=666342067d8f083c720d5df5244fe3596f4b0c400c8bdaf332ebf46ae057031f
  Stored in directory: /root/.cache/pip/wheels/5a/c4/e7/37b9d4a30e03d404d4e2f9a280deea683d631f370384a7d500
  Created wheel for findlibs: filename=findlibs-0.0.2-py3-none-any.whl size=6560 sha256=20adfeb451a6e4e3c41a2aff48b743c270803d80254df993cd9d4e2e094be5f6
  Stored in directory: /root/.cache/pip/wheels/34/e9/92/2a09d5a307252d22fb8d99b13685144b0419d98c36dba7b1c0
Successfully built eccodes findlibs
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████████████████████████

# Imports, paths, train data and train target

In [ ]:
from HailProject.code.grid_model.model import train_model
import HailProject.code.src.data_processing as dp
import HailProject.code.src.prepare_target as pt

In [ ]:
aerology_path = "drive/MyDrive/hail_data/ERA5_Texas/cutted_aerology"
land_path = "drive/MyDrive/hail_data/ERA5_Texas/land"
runoff_path = "drive/MyDrive/hail_data/ERA5_Texas/runoff_only"
extra_feature_path = "drive/MyDrive/hail_data/ERA5_Texas/land"
target_path = "drive/MyDrive/hail_data/target_files"

In [ ]:
full_train_days = dp.prepare_full_train_data(
    aerology_path,
    land_path,
    runoff_path,
    extra_feature_path,
    one_day=False
)
print("Training data:")
print("dims: (n_days, n_features, lat, long): ", full_train_days.shape)

Ignoring index file 'drive/MyDrive/hail_data/ERA5_Texas/runoff_only/adaptor.mars.internal-1657649580.0889146-9595-8-628a165f-5da8-4aad-a03f-d81307c30dbc.grib.923a8.idx' incompatible with GRIB file
Ignoring index file 'drive/MyDrive/hail_data/ERA5_Texas/cutted_aerology/adaptor.mars.internal-1657555514.9637768-23951-18-63c4f4e2-a2b7-47ca-927e-962f3990f7ea.grib.923a8.idx' incompatible with GRIB file


Training data:
dims: (n_days, n_features, lat, long):  (2192, 42, 41, 65)


In [ ]:
target_grid = pt.prepare_target_grid(target_path, (27., 37.), (-109, -93)) 
print("Training target")
print("dims: (n_days, lat, long): ", target_grid.shape)

Training target
dims: (n_days, lat, long):  (2192, 41, 65)


In [ ]:
# Save train data and target
import numpy as np

np.save("drive/MyDrive/hail_data/for_experiments/target_grid.npy", target_grid)
np.save("drive/MyDrive/hail_data/for_experiments/full_train_days.npy", full_train_days)

In [9]:
# Load train data and target
import numpy as np

target_grid = np.load("drive/MyDrive/hail_data/for_experiments/target_grid.npy")
full_train_days = np.load("drive/MyDrive/hail_data/for_experiments/full_train_days.npy")
print("Training data:")
print("dims: (n_days, n_features, lat, long): ", full_train_days.shape)
print("Training target")
print("dims: (n_days, lat, long): ", target_grid.shape)

Training data:
dims: (n_days, n_features, lat, long):  (2192, 42, 41, 65)
Training target
dims: (n_days, lat, long):  (2192, 41, 65)


# Train model

In [58]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

def train_model(train_data, target_grid, new_train: bool = True, model = None):
    if new_train:
        model = CatBoostClassifier(
            iterations=1000,
            learning_rate=0.01,
            task_type="GPU",
            scale_pos_weight=100
        )
    x_train = train_data.reshape(-1, train_data.shape[1])
    y_train = target_grid.reshape(-1)
    model.fit(x_train, y_train)
    return model
x_train, y_train = full_train_days[:1800], target_grid[:1800]
x_val, y_val = full_train_days[1800:], target_grid[1800:]
model = train_model(x_train, y_train)
x_val = x_val.reshape(-1, x_val.shape[1])
y_val = y_val.reshape(-1)
preds_to_val = model.predict(x_val)
print(classification_report(y_val, preds_to_val))

0:	learn: 0.6843892	total: 80.6ms	remaining: 1m 20s
1:	learn: 0.6756128	total: 163ms	remaining: 1m 21s
2:	learn: 0.6672767	total: 239ms	remaining: 1m 19s
3:	learn: 0.6588895	total: 296ms	remaining: 1m 13s
4:	learn: 0.6508204	total: 348ms	remaining: 1m 9s
5:	learn: 0.6430563	total: 398ms	remaining: 1m 5s
6:	learn: 0.6351036	total: 451ms	remaining: 1m 3s
7:	learn: 0.6279900	total: 510ms	remaining: 1m 3s
8:	learn: 0.6210808	total: 560ms	remaining: 1m 1s
9:	learn: 0.6143178	total: 607ms	remaining: 1m
10:	learn: 0.6074015	total: 660ms	remaining: 59.3s
11:	learn: 0.6007696	total: 718ms	remaining: 59.1s
12:	learn: 0.5945468	total: 765ms	remaining: 58.1s
13:	learn: 0.5884755	total: 823ms	remaining: 58s
14:	learn: 0.5825517	total: 870ms	remaining: 57.1s
15:	learn: 0.5767327	total: 922ms	remaining: 56.7s
16:	learn: 0.5710939	total: 970ms	remaining: 56.1s
17:	learn: 0.5655113	total: 1.02s	remaining: 55.6s
18:	learn: 0.5601320	total: 1.07s	remaining: 55.2s
19:	learn: 0.5549092	total: 1.12s	remaini

In [55]:
preds_to_val.sum()

492035.0

In [51]:
print(classification_report(y_train.reshape(-1), preds_to_val))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00   4790655
         1.0       0.38      0.16      0.23      6345

    accuracy                           1.00   4797000
   macro avg       0.69      0.58      0.61   4797000
weighted avg       1.00      1.00      1.00   4797000



In [50]:
precision_score(y_train.reshape(-1), preds_to_val)

0.38104020656584286

In [49]:
from sklearn.metrics import precision_score
preds_to_val = model.predict(x_train.reshape(-1, x_val.shape[1]))
precision_score(y_train.reshape(-1), preds_to_val)

ValueError: ignored

In [41]:
y_val.sum()

1073.0

In [ ]:
model.save_model("/content/drive/MyDrive/hail_data/model/model")

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier()
model.load_model("/content/drive/MyDrive/hail_data/model/model")

In [59]:
import glob
dataset_path = "/content/drive/MyDrive/hail_data/CMIP/np_cmips_rcp85_r1i1p1"
def check_leap_year(year):
    if (year % 400 == 0) and (year % 100 == 0):
        return True
    elif (year % 4 ==0) and (year % 100 != 0):
        return True
    else:
        return False

def inference_model(model, dataset_path):
    paths = sorted(glob.glob(dataset_path + "/*.npy"))
    month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    leap_month = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    full_preds = []
    years = [year for year in range(2022, 2051)]
    for path, year in zip(paths, years):
        data = np.load(path)
        x_test = data.reshape(-1, data.shape[1])
        preds = model.predict_proba(x_test)[:, 1] 
        preds = preds.reshape((-1, data.shape[2], data.shape[3]), order="F")
        if check_leap_year(year):
            for i, days_in_month in enumerate(month):
                full_preds.append(np.expand_dims(np.max(preds[sum(month[:i]):sum(month[:i]) + days_in_month][2][16], axis=0), axis=0))
        else:
            for i, days_in_month in enumerate(leap_month):
                full_preds.append(np.expand_dims(np.max(preds[sum(month[:i]):sum(month[:i]) + days_in_month][2][16], axis=0), axis=0))
        print("Forecasted year: ", year)
    full_preds = np.concatenate(full_preds, axis=0)
    return full_preds

preds = inference_model(model, dataset_path)

Forecasted year:  2022
Forecasted year:  2023
Forecasted year:  2024
Forecasted year:  2025
Forecasted year:  2026
Forecasted year:  2027
Forecasted year:  2028
Forecasted year:  2029
Forecasted year:  2030
Forecasted year:  2031
Forecasted year:  2032
Forecasted year:  2033
Forecasted year:  2034
Forecasted year:  2035
Forecasted year:  2036
Forecasted year:  2037
Forecasted year:  2038
Forecasted year:  2039
Forecasted year:  2040
Forecasted year:  2041
Forecasted year:  2042
Forecasted year:  2043
Forecasted year:  2044
Forecasted year:  2045
Forecasted year:  2046
Forecasted year:  2047
Forecasted year:  2048
Forecasted year:  2049
Forecasted year:  2050


In [12]:
np.save("drive/MyDrive/hail_data/model/novorosiysk_preds.npy", preds)

In [60]:
def create_dataset(preds):
    months = [f"{month}/{year}" for year in range(2022, 2051) for month in range(1,13)]
    mean = np.mean(preds)
    dataset = pd.DataFrame({'month': months, "risk": preds, "damage": [1 if pred > mean else 0 for pred in preds]})
    return dataset
ds = create_dataset(preds)
ds.to_csv("Novorosiysk_damage.csv")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
preds = np.load("drive/MyDrive/hail_data/model/preds.npy")
for i in range(29):
    plt.figure(figsize=(10, 5))
    plt.imshow(preds[i], cmap='hot', interpolation='nearest')
    plt.show()

In [4]:
preds.shape

(29, 32, 154)

In [ ]:
import xarray as xr
path = "/content/drive/MyDrive/hail_data/CMIP/huss_3hr_rus/huss_3hr_MRI-CGCM3_rcp85_r1i1p1_202201010000-202212312100.nc"
ds = xr.open_dataset(path)
ds

<xarray.Dataset>
Dimensions:   (time: 2920, lat: 32, bnds: 2, lon: 154)
Coordinates:
  * time      (time) datetime64[ns] 2022-01-01 ... 2022-12-31T21:00:00
  * lat       (lat) float64 42.06 43.18 44.3 45.42 ... 73.46 74.58 75.7 76.82
  * lon       (lon) float64 19.12 20.25 21.38 22.5 ... 187.9 189.0 190.1 191.2
    height    float64 ...
Dimensions without coordinates: bnds
Data variables:
    lat_bnds  (lat, bnds) float64 ...
    lon_bnds  (lon, bnds) float64 ...
    huss      (time, lat, lon) float32 ...
Attributes: (12/27)
    institution:            MRI (Meteorological Research Institute, Tsukuba, ...
    institute_id:           MRI
    experiment_id:          rcp85
    source:                 MRI-CGCM3 2011 atmosphere: GSMUV (gsmuv-110112, T...
    model_id:               MRI-CGCM3
    forcing:                GHG, SA, Oz, LU, Sl, Vl, BC, OC (GHG includes CO2...
    ...                     ...
    table_id:               Table 3hr (01 February 2012) f111472c4a12d2f07d41...
    title:                  MRI-CGCM3 model output prepared for CMIP5 RCP8.5
    parent_experiment:      historical
    modeling_realm:         atmos
    realization:            1
    cmor_version:           2.8.1

In [5]:
import pandas as pd
global predictions 
predictions = preds

def cart_prod(x, y, z):
    return np.array([[x0, y0, z0] for x0 in x for y0 in y for z0 in z])

def change_prob(row):
    years = [year for year in range(2022, 2051)]
    lat_grid = [42.05582, 43.17731, 44.29879, 45.42028, 46.54176, 47.66325, 48.78473,
       49.90621, 51.02769, 52.14917, 53.27066, 54.39214, 55.51361, 56.63509,
       57.75657, 58.87804, 59.99952, 61.12099, 62.24246, 63.36393, 64.4854 ,
       65.60686, 66.72833, 67.84978, 68.97124, 70.09269, 71.21414, 72.33558,
       73.45701, 74.57843, 75.69984, 76.82124]
    long_grid = [ 19.125,  20.25 ,  21.375,  22.5  ,  23.625,  24.75 ,  25.875,  27.   ,
        28.125,  29.25 ,  30.375,  31.5  ,  32.625,  33.75 ,  34.875,  36.   ,
        37.125,  38.25 ,  39.375,  40.5  ,  41.625,  42.75 ,  43.875,  45.   ,
        46.125,  47.25 ,  48.375,  49.5  ,  50.625,  51.75 ,  52.875,  54.   ,
        55.125,  56.25 ,  57.375,  58.5  ,  59.625,  60.75 ,  61.875,  63.   ,
        64.125,  65.25 ,  66.375,  67.5  ,  68.625,  69.75 ,  70.875,  72.   ,
        73.125,  74.25 ,  75.375,  76.5  ,  77.625,  78.75 ,  79.875,  81.   ,
        82.125,  83.25 ,  84.375,  85.5  ,  86.625,  87.75 ,  88.875,  90.   ,
        91.125,  92.25 ,  93.375,  94.5  ,  95.625,  96.75 ,  97.875,  99.   ,
       100.125, 101.25 , 102.375, 103.5  , 104.625, 105.75 , 106.875, 108.   ,
       109.125, 110.25 , 111.375, 112.5  , 113.625, 114.75 , 115.875, 117.   ,
       118.125, 119.25 , 120.375, 121.5  , 122.625, 123.75 , 124.875, 126.   ,
       127.125, 128.25 , 129.375, 130.5  , 131.625, 132.75 , 133.875, 135.   ,
       136.125, 137.25 , 138.375, 139.5  , 140.625, 141.75 , 142.875, 144.   ,
       145.125, 146.25 , 147.375, 148.5  , 149.625, 150.75 , 151.875, 153.   ,
       154.125, 155.25 , 156.375, 157.5  , 158.625, 159.75 , 160.875, 162.   ,
       163.125, 164.25 , 165.375, 166.5  , 167.625, 168.75 , 169.875, 171.   ,
       172.125, 173.25 , 174.375, 175.5  , 176.625, 177.75 , 178.875, 180.   ,
       181.125, 182.25 , 183.375, 184.5  , 185.625, 186.75 , 187.875, 189.   ,
       190.125, 191.25 ]
    lat_to_idx = {}.fromkeys(lat_grid)
    long_to_idx = {}.fromkeys(lat_grid)
    year_to_idx = {}.fromkeys(years)

    lat_to_idx = {}.fromkeys(lat_grid)
    long_to_idx = {}.fromkeys(lat_grid)
    year_to_idx = {}.fromkeys(years)

    for i, lat_ in enumerate(lat_grid):
        lat_to_idx[lat_] = i
    for j, long_ in enumerate(long_grid):
        long_to_idx[long_] = j
    for k, year in enumerate(years):
        year_to_idx[year] = k
    row[3] = preds[year_to_idx[row[0]],
                   lat_to_idx[row[2]],
                   long_to_idx[row[1]]]
    return row

def create_dataset():
    years = [year for year in range(2022, 2051)]
    lat_grid = [42.05582, 43.17731, 44.29879, 45.42028, 46.54176, 47.66325, 48.78473,
       49.90621, 51.02769, 52.14917, 53.27066, 54.39214, 55.51361, 56.63509,
       57.75657, 58.87804, 59.99952, 61.12099, 62.24246, 63.36393, 64.4854 ,
       65.60686, 66.72833, 67.84978, 68.97124, 70.09269, 71.21414, 72.33558,
       73.45701, 74.57843, 75.69984, 76.82124]
    long_grid = [ 19.125,  20.25 ,  21.375,  22.5  ,  23.625,  24.75 ,  25.875,  27.   ,
        28.125,  29.25 ,  30.375,  31.5  ,  32.625,  33.75 ,  34.875,  36.   ,
        37.125,  38.25 ,  39.375,  40.5  ,  41.625,  42.75 ,  43.875,  45.   ,
        46.125,  47.25 ,  48.375,  49.5  ,  50.625,  51.75 ,  52.875,  54.   ,
        55.125,  56.25 ,  57.375,  58.5  ,  59.625,  60.75 ,  61.875,  63.   ,
        64.125,  65.25 ,  66.375,  67.5  ,  68.625,  69.75 ,  70.875,  72.   ,
        73.125,  74.25 ,  75.375,  76.5  ,  77.625,  78.75 ,  79.875,  81.   ,
        82.125,  83.25 ,  84.375,  85.5  ,  86.625,  87.75 ,  88.875,  90.   ,
        91.125,  92.25 ,  93.375,  94.5  ,  95.625,  96.75 ,  97.875,  99.   ,
       100.125, 101.25 , 102.375, 103.5  , 104.625, 105.75 , 106.875, 108.   ,
       109.125, 110.25 , 111.375, 112.5  , 113.625, 114.75 , 115.875, 117.   ,
       118.125, 119.25 , 120.375, 121.5  , 122.625, 123.75 , 124.875, 126.   ,
       127.125, 128.25 , 129.375, 130.5  , 131.625, 132.75 , 133.875, 135.   ,
       136.125, 137.25 , 138.375, 139.5  , 140.625, 141.75 , 142.875, 144.   ,
       145.125, 146.25 , 147.375, 148.5  , 149.625, 150.75 , 151.875, 153.   ,
       154.125, 155.25 , 156.375, 157.5  , 158.625, 159.75 , 160.875, 162.   ,
       163.125, 164.25 , 165.375, 166.5  , 167.625, 168.75 , 169.875, 171.   ,
       172.125, 173.25 , 174.375, 175.5  , 176.625, 177.75 , 178.875, 180.   ,
       181.125, 182.25 , 183.375, 184.5  , 185.625, 186.75 , 187.875, 189.   ,
       190.125, 191.25 ]

    np_ds = cart_prod(years, long_grid, lat_grid)
    dataset = pd.DataFrame({'year': np_ds[:, 0], 'lon': np_ds[:, 1], 'lat': np_ds[:, 2]})
    dataset["Probability_of_hail"] = None
    dataset = dataset.apply(change_prob, axis = 1) 
    return dataset

In [ ]:
lat_idx = 2
long_idx = 16

In [7]:
lat_grid = [42.05582, 43.17731, 44.29879, 45.42028, 46.54176, 47.66325, 48.78473,
    49.90621, 51.02769, 52.14917, 53.27066, 54.39214, 55.51361, 56.63509,
    57.75657, 58.87804, 59.99952, 61.12099, 62.24246, 63.36393, 64.4854 ,
    65.60686, 66.72833, 67.84978, 68.97124, 70.09269, 71.21414, 72.33558,
    73.45701, 74.57843, 75.69984, 76.82124]
long_grid = [ 19.125,  20.25 ,  21.375,  22.5  ,  23.625,  24.75 ,  25.875,  27.   ,
    28.125,  29.25 ,  30.375,  31.5  ,  32.625,  33.75 ,  34.875,  36.   ,
    37.125,  38.25 ,  39.375,  40.5  ,  41.625,  42.75 ,  43.875,  45.   ,
    46.125,  47.25 ,  48.375,  49.5  ,  50.625,  51.75 ,  52.875,  54.   ,
    55.125,  56.25 ,  57.375,  58.5  ,  59.625,  60.75 ,  61.875,  63.   ,
    64.125,  65.25 ,  66.375,  67.5  ,  68.625,  69.75 ,  70.875,  72.   ,
    73.125,  74.25 ,  75.375,  76.5  ,  77.625,  78.75 ,  79.875,  81.   ,
    82.125,  83.25 ,  84.375,  85.5  ,  86.625,  87.75 ,  88.875,  90.   ,
    91.125,  92.25 ,  93.375,  94.5  ,  95.625,  96.75 ,  97.875,  99.   ,
    100.125, 101.25 , 102.375, 103.5  , 104.625, 105.75 , 106.875, 108.   ,
    109.125, 110.25 , 111.375, 112.5  , 113.625, 114.75 , 115.875, 117.   ,
    118.125, 119.25 , 120.375, 121.5  , 122.625, 123.75 , 124.875, 126.   ,
    127.125, 128.25 , 129.375, 130.5  , 131.625, 132.75 , 133.875, 135.   ,
    136.125, 137.25 , 138.375, 139.5  , 140.625, 141.75 , 142.875, 144.   ,
    145.125, 146.25 , 147.375, 148.5  , 149.625, 150.75 , 151.875, 153.   ,
    154.125, 155.25 , 156.375, 157.5  , 158.625, 159.75 , 160.875, 162.   ,
    163.125, 164.25 , 165.375, 166.5  , 167.625, 168.75 , 169.875, 171.   ,
    172.125, 173.25 , 174.375, 175.5  , 176.625, 177.75 , 178.875, 180.   ,
    181.125, 182.25 , 183.375, 184.5  , 185.625, 186.75 , 187.875, 189.   ,
    190.125, 191.25 ]
lat_to_idx = {}.fromkeys(lat_grid)
long_to_idx = {}.fromkeys(lat_grid)
for i, lat_ in enumerate(lat_grid):
    lat_to_idx[lat_] = i
for j, long_ in enumerate(long_grid):
    long_to_idx[long_] = j

In [ ]:
preds.shape

(29, 32, 154)

In [ ]:
dataset_for_drawing = create_dataset()

In [ ]:
dataset_for_drawing.to_csv("dataset_for_drawing.csv")

In [ ]:
def cart_prod(x, y, z):
    return np.array([[x0, y0, z0] for x0 in x for y0 in y for z0 in z])
  
cart_prod([2019,2020,2021], [-30, -29], [101, 102, 103, 104])

array([[2019,  -30,  101],
       [2019,  -30,  102],
       [2019,  -30,  103],
       [2019,  -30,  104],
       [2019,  -29,  101],
       [2019,  -29,  102],
       [2019,  -29,  103],
       [2019,  -29,  104],
       [2020,  -30,  101],
       [2020,  -30,  102],
       [2020,  -30,  103],
       [2020,  -30,  104],
       [2020,  -29,  101],
       [2020,  -29,  102],
       [2020,  -29,  103],
       [2020,  -29,  104],
       [2021,  -30,  101],
       [2021,  -30,  102],
       [2021,  -30,  103],
       [2021,  -30,  104],
       [2021,  -29,  101],
       [2021,  -29,  102],
       [2021,  -29,  103],
       [2021,  -29,  104]])

In [ ]:
%matplotlib notebook

import random
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import matplotlib.animation as animation


fps = 29
nSeconds = 1
snapshots = preds#[ np.random.rand(5,5) for _ in range( nSeconds * fps ) ]

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure( figsize=(8,8) )

a = snapshots[0]
im = plt.imshow(a, interpolation='none', aspect='auto', vmin=0, vmax=1)

def animate_func(i):
    if i % fps == 0:
        print( '.', end ='' )

    im.set_array(snapshots[i])
    return [im]

anim = animation.FuncAnimation(
                               fig, 
                               animate_func, 
                               frames = nSeconds * fps,
                               interval = 1000 / fps, # in ms
                               )

anim.save('test_anim.mp4', fps=fps, extra_args=['-vcodec', 'libx264'])

print('Done!')

<IPython.core.display.Javascript object>

..Done!


<IPython.core.display.Javascript object>

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/matplotlib/cbook/__init__.py", line 196, in process
    func(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/matplotlib/animation.py", line 1467, in _stop
    self.event_source.remove_callback(self._loop_delay)
AttributeError: 'NoneType' object has no attribute 'remove_callback'


In [ ]:
preds[1]

array([[3.67124597, 0.16172262, 3.35154734, ..., 4.3792858 , 5.08670714,
        4.73635848],
       [4.38372192, 4.47603176, 5.12921365, ..., 5.15506953, 5.09766968,
        5.25595249],
       [5.29402409, 5.12113297, 5.15955582, ..., 4.95237722, 4.68154508,
        4.91521509],
       ...,
       [0.24447244, 0.24971228, 0.2541301 , ..., 0.24894217, 0.24851136,
        0.24322008],
       [0.24052848, 0.24788598, 0.24879681, ..., 0.25101164, 0.25115292,
        0.24993995],
       [0.24953828, 0.25272435, 0.25140116, ..., 0.2512241 , 0.25197343,
        0.25243022]])